In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk #Image Processing
import pytesseract
from gtts import gTTS
from googletrans import Translator, LANGUAGES
import cv2
import os


class OCRApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image To Text And Text To Speech")
        self.root.geometry("1000x800")

        self.image_path = None
        self.text = ""
        self.translated_text = ""

        # Header: here we can use header file which includes heading of file
        self.header_frame = tk.Frame(root, bg="#00BFFF", pady=20)
        self.header_frame.pack(fill=tk.X)
        self.title_label = tk.Label(self.header_frame, text="Image To Text And Text to Speech",
                                    font=("Helvetica", 32, "bold"), fg="white", bg="#00BFFF")
        self.title_label.pack()

        # Button frame:Size of the button
        self.button_frame = tk.Frame(root, bg="#F4F4F9")
        self.button_frame.pack(pady=30)

        self.upload_button = self.create_custom_button("Upload Image", self.upload_image, "#32CD32")
        self.upload_button.pack(pady=10)

        self.capture_button = self.create_custom_button("Capture Image", self.capture_image, "#FFA500")
        self.capture_button.pack(pady=10)

        self.text_to_speech_button = self.create_custom_button("Convert to Speech", self.text_to_speech, "#FF6347")
        self.text_to_speech_button.pack(pady=10)

        self.exit_button = self.create_custom_button("Exit", self.exit_application, "#DC143C")
        self.exit_button.pack(pady=10)

    def create_custom_button(self, text, command, color):
        """Created a custom button with hover effects."""
        button = tk.Button(
            self.button_frame,
            text=text,
            command=command,
            width=20,
            height=2,
            font=("Helvetica", 18),
            bg=color,
            fg="white",
            relief="solid",
            bd=2,
            padx=10,
            pady=10,
            activebackground="#006400"
        )
        button.bind("<Enter>", lambda e: self.on_hover_in(button, color))
        button.bind("<Leave>", lambda e: self.on_hover_out(button, color))
        return button

    def on_hover_in(self, button, color):
        button.config(bg="white", fg=color)

    def on_hover_out(self, button, color):
        button.config(bg=color, fg="white")

    def upload_image(self):
        self.image_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")])
        if self.image_path:
            self.process_image(self.image_path)

    # def capture_image(self):
    #     cap = cv2.VideoCapture(0)
    #     if not cap.isOpened():
    #         messagebox.showerror("Error", "Failed to access the webcam.")
    #         return

    #     capture_window = tk.Toplevel(self.root)
    #     capture_window.title("Webcam - Press 's' to Capture")
    #     capture_window.geometry("640x480")

    #     label = tk.Label(capture_window)
    #     label.pack()

    #     def show_frame():
    #         ret, frame = cap.read()
    #         if ret:
    #             frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #             img = Image.fromarray(frame)
    #             imgtk = ImageTk.PhotoImage(image=img)
    #             label.imgtk = imgtk
    #             label.configure(image=imgtk)
    #         label.after(10, show_frame)

    #     def capture_image():
    #         ret, frame = cap.read()
    #         if ret:
    #             self.image_path = "captured_image.jpg"
    #             cv2.imwrite(self.image_path, frame)
    #             messagebox.showinfo("Info", f"Image saved to {self.image_path}")
    #             capture_window.destroy()
    #             cap.release()
    #             self.process_image(self.image_path)

    #     capture_window.bind('<s>', lambda event: capture_image())
    #     capture_window.protocol("WM_DELETE_WINDOW", lambda: (capture_window.destroy(), cap.release()))
    #     show_frame()
    def capture_image(self):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            messagebox.showerror("Error", "Failed to access the webcam.")
            return

        capture_window = tk.Toplevel(self.root)
        capture_window.title("Webcam - Press 's' to Capture")
        capture_window.geometry("640x480")

        label = tk.Label(capture_window)
        label.pack()

        def show_frame():
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame)
                imgtk = ImageTk.PhotoImage(image=img)
                label.imgtk = imgtk
                label.configure(image=imgtk)
            label.after(10, show_frame)

        def capture_image():
            ret, frame = cap.read()
            if ret:
                self.image_path = "captured_image.jpg"
                cv2.imwrite(self.image_path, frame)
                messagebox.showinfo("Info", f"Image saved to {self.image_path}")
                capture_window.destroy()
                cap.release()
                self.process_image(self.image_path)

        capture_window.bind('<s>', lambda event: capture_image())
        capture_window.protocol("WM_DELETE_WINDOW", lambda: (capture_window.destroy(), cap.release()))
        show_frame()

    def process_image(self, image_path):
        try:
            image = Image.open(image_path)
            self.text = pytesseract.image_to_string(image)
            self.show_text()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to process image: {e}")

    def show_text(self):
        if not self.text.strip():
            messagebox.showinfo("Info", "No text found in the image.")
            return

        text_window = tk.Toplevel(self.root)
        text_window.title("Extracted Text")
        text_window.geometry("600x500")

        text_label = tk.Label(text_window, text="Extracted Text", font=("Helvetica", 24), bg="#F4F4F9", fg="#333333")
        text_label.pack(pady=10)

        text_area = tk.Text(text_window, wrap=tk.WORD, font=("Helvetica", 16), bg="lightgray", fg="black", height=10)
        text_area.pack(expand=True, fill=tk.BOTH)
        text_area.insert(tk.END, self.text)
        text_area.config(state=tk.DISABLED)

        # Language selection for translation uses google ocr
        self.lang_var = tk.StringVar()
        self.lang_dropdown = ttk.Combobox(text_window, textvariable=self.lang_var, font=("Helvetica", 14))
        self.lang_dropdown['values'] = list(LANGUAGES.values())
        self.lang_dropdown.pack(pady=10)
        self.lang_dropdown.set("english")  # Default selection is English

        # Translation button uses google ocr
        translate_button = tk.Button(
            text_window,
            text="Translate Text",
            command=self.translate_text,
            font=("Helvetica", 16),
            bg="#1E90FF",
            fg="white",
            padx=10,
            pady=10
        )
        translate_button.pack(pady=10)

        # Convert to speech button
        speech_button = tk.Button(
            text_window,
            text="Convert to Speech",
            command=self.text_to_speech_in_translation,
            font=("Helvetica", 16),
            bg="#FF6347",
            fg="white",
            padx=10,
            pady=10
        )
        speech_button.pack(pady=10)

    def translate_text(self):
        if not self.text.strip():
            messagebox.showinfo("Info", "No text to translate.")
            return

        target_lang = self.lang_var.get()
        if not target_lang:
            messagebox.showwarning("Warning", "Please select a target language.")
            return

        # Find language code
        target_lang_code = [code for code, name in LANGUAGES.items() if name == target_lang]
        if not target_lang_code:
            messagebox.showerror("Error", "Invalid language selected.")
            return

        try:
            translator = Translator()
            self.translated_text = translator.translate(self.text, dest=target_lang_code[0]).text
            self.show_translated_text()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to translate text: {e}")

    def show_translated_text(self):
        text_window = tk.Toplevel(self.root)
        text_window.title("Translated Text")
        text_window.geometry("600x400")

        text_label = tk.Label(text_window, text="Translated Text", font=("Helvetica", 24), bg="#F4F4F9", fg="#333333")
        text_label.pack(pady=10)

        text_area = tk.Text(text_window, wrap=tk.WORD, font=("Helvetica", 16), bg="lightgray", fg="black", height=10)
        text_area.pack(expand=True, fill=tk.BOTH)
        text_area.insert(tk.END, self.translated_text)
        text_area.config(state=tk.DISABLED)

    def text_to_speech(self):
        text_to_read = self.text

        if not text_to_read.strip():
            messagebox.showinfo("Info", "No text to convert to speech.")
            return

        try:
            tts = gTTS(text=text_to_read, lang="en", slow=False)
            audio_file = "output_audio.mp3"
            tts.save(audio_file)
            messagebox.showinfo("Success", f"Audio saved as {audio_file}")
            os.system(f"start {audio_file}" if os.name == "nt" else f"xdg-open {audio_file}")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to convert text to speech: {e}")

    def text_to_speech_in_translation(self):
        text_to_read = self.translated_text if self.translated_text else self.text

        if not text_to_read.strip():
            messagebox.showinfo("Info", "No text to convert to speech.")
            return

        target_lang = self.lang_var.get()
        if not target_lang:
            messagebox.showwarning("Warning", "Please select a language for speech.")
            return

        # Find language code for speech
        target_lang_code = [code for code, name in LANGUAGES.items() if name == target_lang]
        if not target_lang_code:
            messagebox.showerror("Error", "Invalid language selected for speech.")
            return

        try:
            tts = gTTS(text=text_to_read, lang=target_lang_code[0], slow=False)
            audio_file = "output_audio.mp3"
            tts.save(audio_file)
            messagebox.showinfo("Success", f"Audio saved as {audio_file}")
            os.system(f"start {audio_file}" if os.name == "nt" else f"xdg-open {audio_file}")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to convert text to speech: {e}")

    def exit_application(self):
        """Properly closes the application."""
        self.root.destroy()  # Ensures the window is closed


if __name__ == "__main__":
    root = tk.Tk()
    app = OCRApp(root)
    root.protocol("WM_DELETE_WINDOW", root.destroy)  
    root.mainloop()



In [6]:
import sys
print(sys.version)


3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]


In [7]:
pip install tensorflow==2.10


   ---------------------------------------- 0.0/455.9 MB ? eta -:--:--
    --------------------------------------- 6.0/455.9 MB 37.0 MB/s eta 0:00:13
   - -------------------------------------- 13.1/455.9 MB 34.3 MB/s eta 0:00:13
   - -------------------------------------- 19.1/455.9 MB 32.7 MB/s eta 0:00:14
   -- ------------------------------------- 25.4/455.9 MB 32.9 MB/s eta 0:00:14
   -- ------------------------------------- 31.2/455.9 MB 31.4 MB/s eta 0:00:14
   --- ------------------------------------ 38.8/455.9 MB 32.0 MB/s eta 0:00:14
   --- ------------------------------------ 45.4/455.9 MB 32.1 MB/s eta 0:00:13
   ---- ----------------------------------- 52.4/455.9 MB 32.7 MB/s eta 0:00:13
   ----- ---------------------------------- 59.5/455.9 MB 32.7 MB/s eta 0:00:13
   ----- ---------------------------------- 67.4/455.9 MB 33.0 MB/s eta 0:00:12
   ------ --------------------------------- 74.7/455.9 MB 33.6 MB/s eta 0:00:12
   ------- -------------------------------- 80.7/4

In [2]:
pip install pandas

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.6 MB 8.3 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/11.6 MB 7.9 MB/s eta 0:00:02
   ---------------- ----------------------- 4.7/11.6 MB 9.2 MB/s eta 0:00:01
   ------------------------------ --------- 8.9/11.6 MB 12.3 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 12.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [26]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk
import pytesseract
from sklearn.metrics import accuracy_score, confusion_matrix
import os


class OCRApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image To Text And Text To Speech")
        self.root.geometry("1000x800")

        self.image_path = None
        self.text = ""
        self.ground_truth = ""

        # Header
        self.header_frame = tk.Frame(root, bg="#00BFFF", pady=20)
        self.header_frame.pack(fill=tk.X)
        self.title_label = tk.Label(self.header_frame, text="Image To Text And Accuracy Checker",
                                    font=("Helvetica", 32, "bold"), fg="white", bg="#00BFFF")
        self.title_label.pack()

        # Button frame
        self.button_frame = tk.Frame(root, bg="#F4F4F9")
        self.button_frame.pack(pady=30)

        self.upload_button = self.create_custom_button("Upload Image", self.upload_image, "#32CD32")
        self.upload_button.pack(pady=10)

        self.ground_truth_button = self.create_custom_button("Set Ground Truth", self.set_ground_truth_window, "#1E90FF")
        self.ground_truth_button.pack(pady=10)

        self.accuracy_button = self.create_custom_button("Check Accuracy", self.calculate_accuracy, "#FFA500")
        self.accuracy_button.pack(pady=10)

        self.exit_button = self.create_custom_button("Exit", self.exit_application, "#DC143C")
        self.exit_button.pack(pady=10)

    def create_custom_button(self, text, command, color):
        """Create a custom button with hover effects."""
        button = tk.Button(
            self.button_frame,
            text=text,
            command=command,
            width=20,
            height=2,
            font=("Helvetica", 18),
            bg=color,
            fg="white",
            relief="solid",
            bd=2,
            padx=10,
            pady=10,
            activebackground="#006400"
        )
        button.bind("<Enter>", lambda e: self.on_hover_in(button, color))
        button.bind("<Leave>", lambda e: self.on_hover_out(button, color))
        return button

    def on_hover_in(self, button, color):
        button.config(bg="white", fg=color)

    def on_hover_out(self, button, color):
        button.config(bg=color, fg="white")

    def upload_image(self):
        self.image_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")])
        if self.image_path:
            self.process_image(self.image_path)

    def process_image(self, image_path):
        try:
            image = Image.open(image_path)
            self.text = pytesseract.image_to_string(image).strip()
            messagebox.showinfo("Info", "Image processed successfully! Text extracted.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to process image: {e}")

    def set_ground_truth_window(self):
        """Open a window to set the ground truth text."""
        self.ground_truth_window = tk.Toplevel(self.root)
        self.ground_truth_window.title("Set Ground Truth")
        self.ground_truth_window.geometry("600x400")

        # Label
        label = tk.Label(self.ground_truth_window, text="Enter Ground Truth Text:", font=("Helvetica", 16))
        label.pack(pady=10)

        # Text area
        self.ground_truth_text = tk.Text(self.ground_truth_window, wrap=tk.WORD, font=("Helvetica", 14), height=10)
        self.ground_truth_text.pack(expand=True, fill=tk.BOTH, padx=10, pady=10)

        # Button to save and close
        save_button = tk.Button(
            self.ground_truth_window,
            text="Save and Close",
            command=self.save_and_close_ground_truth,
            font=("Helvetica", 14),
            bg="#32CD32",
            fg="white",
            padx=10,
            pady=5
        )
        save_button.pack(pady=10)

    def save_and_close_ground_truth(self):
        """Save the ground truth text and close the window."""
        self.ground_truth = self.ground_truth_text.get("1.0", tk.END).strip()
        self.ground_truth_window.destroy()
        messagebox.showinfo("Info", "Ground Truth Text Saved Successfully")

    def calculate_accuracy(self):
        """Calculate accuracy and confusion matrix."""
        if not self.text or not self.ground_truth:
            messagebox.showwarning("Warning", "Please process an image and set ground truth first.")
            return

        # Preprocess the extracted and ground truth text for comparison
        extracted_words = self.text.split()
        ground_truth_words = self.ground_truth.split()

        # Ensure both lists are of the same length
        if len(extracted_words) != len(ground_truth_words):
            messagebox.showwarning("Warning", "The extracted text and ground truth have different lengths.")
            return

        # Calculate accuracy
        accuracy = accuracy_score(ground_truth_words, extracted_words)

        # Calculate confusion matrix
        confusion = confusion_matrix(ground_truth_words, extracted_words, labels=list(set(ground_truth_words)))

        # Display the results
        result_window = tk.Toplevel(self.root)
        result_window.title("Accuracy and Confusion Matrix")
        result_window.geometry("600x400")

        accuracy_label = tk.Label(result_window, text=f"Accuracy: {accuracy:.2%}", font=("Helvetica", 16), fg="green")
        accuracy_label.pack(pady=10)

        confusion_label = tk.Label(result_window, text="Confusion Matrix:", font=("Helvetica", 16))
        confusion_label.pack(pady=10)

        confusion_text = tk.Text(result_window, wrap=tk.WORD, font=("Helvetica", 14), height=10)
        confusion_text.pack(expand=True, fill=tk.BOTH, padx=10, pady=10)
        confusion_text.insert(tk.END, str(confusion))
        confusion_text.config(state=tk.DISABLED)

    def exit_application(self):
        """Properly closes the application."""
        self.root.destroy()  # Ensures the window is closed


if __name__ == "__main__":
    root = tk.Tk()
    app = OCRApp(root)
    root.protocol("WM_DELETE_WINDOW", root.destroy)  # Properly closes when the window is closed
    root.mainloop()


In [38]:
from sklearn.metrics import confusion_matrix
from rapidfuzz import fuzz
import numpy as np


def calculate_accuracy_and_confusion_matrix(ground_truth, extracted_text):
    # Normalize texts (remove extra spaces, convert to lowercase, etc.)
    ground_truth = " ".join(ground_truth.replace("\n", " ").strip().lower().split())
    extracted_text = " ".join(extracted_text.replace("\n", " ").strip().lower().split())

    # Generate character-level comparison
    max_len = max(len(ground_truth), len(extracted_text))
    padded_ground_truth = ground_truth.ljust(max_len)
    padded_extracted_text = extracted_text.ljust(max_len)

    # Create labels for characters
    all_characters = sorted(set(padded_ground_truth + padded_extracted_text))
    char_to_index = {char: idx for idx, char in enumerate(all_characters)}

    # Map ground truth and extracted text to indices
    ground_truth_indices = [char_to_index[char] for char in padded_ground_truth]
    extracted_text_indices = [char_to_index[char] for char in padded_extracted_text]

    # Compute character-level accuracy
    matches = sum(gt == et for gt, et in zip(ground_truth_indices, extracted_text_indices))
    char_accuracy = matches / max_len * 100

    # Compute word-level accuracy
    ground_truth_words = ground_truth.split()
    extracted_words = extracted_text.split()
    word_accuracy = fuzz.ratio(ground_truth_words, extracted_words)

    # Compute confusion matrix
    cm = confusion_matrix(ground_truth_indices, extracted_text_indices, labels=range(len(all_characters)))

    return char_accuracy, word_accuracy, cm, all_characters


# Example usage
ground_truth_text = """Paste your ground truth text here"""
extracted_text = """Paste your extracted text here"""
char_accuracy, word_accuracy, cm, characters = calculate_accuracy_and_confusion_matrix(
    ground_truth_text, extracted_text
)

print(f"Character-Level Accuracy: {char_accuracy:.2f}%")
print(f"Word-Level Accuracy: {word_accuracy:.2f}%")
print("Confusion Matrix:")
print(cm)
print("Characters:", characters)



Character-Level Accuracy: 39.39%
Word-Level Accuracy: 72.73%
Confusion Matrix:
[[2 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [3 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0 3 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
Characters: [' ', 'a', 'c', 'd', 'e', 'g', 'h', 'n', 'o', 'p', 'r', 's', 't', 'u', 'x', 'y']


In [36]:
pip install rapidfuzz


   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 17.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [59]:
pip install webcolors


In [8]:
!pip install opencv-python-headless




  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)


In [10]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [41]:
pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 7.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17457 sha256=6349cb9a35287ed396f231cea9ef91da70857ef3de3f5c875a6c4eb3c74d1e14
  Stored in directory: c:\users\suhas\appdata\local\pip\cache\wheels\95\0f\04\b17a72024b56a60e499ce1a6313d283ed5ba332407155bee03
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled id

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.2.5 requires httpx>=0.25.0, but you have httpx 0.13.3 which is incompatible.
tensorflow-intel 2.18.0 requires keras>=3.5.0, but you have keras 2.15.0 which is incompatible.


In [45]:
pip install googletrans==4.0.0-rc1 pillow pytesseract gtts opencv-python-headless


Note: you may need to restart the kernel to use updated packages.


In [45]:
pip install googletrans==4.0.0-rc1 pillow pytesseract gtts opencv-python-headless


Note: you may need to restart the kernel to use updated packages.


In [49]:
pip install googletrans==4.0.0-rc1 pillow pytesseract gtts opencv-python-headless


Note: you may need to restart the kernel to use updated packages.
